In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('supermarket-database.csv')
data.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [21]:
from copy import deepcopy
df = deepcopy(data[['Branch', 'City']])
df.Branch = df.Branch.map({'A':'1','C':'2','B':3})
df.City = df.City.map({'Yangon':'1','Naypyitaw':'2','Mandalay':3})
df.drop_duplicates(inplace=True)
df

,Branch,City
0,1,1
1,2,2
9,3,3


In [64]:

sale = deepcopy(data)
sale['customer_id'] = None
sale.loc[(sale['Customer type']=='Member')&(sale['Gender']=='Female'), 'customer_id'] = 1
sale.loc[(sale['Customer type']=='Member')&(sale['Gender']=='Male'), 'customer_id'] = 2
sale.loc[(sale['Customer type']=='Normal')&(sale['Gender']=='Female'), 'customer_id'] = 3
sale.loc[(sale['Customer type']=='Normal')&(sale['Gender']=='Male'), 'customer_id'] = 4
sale['store_id'] = None
def store_id(x):
    if x == 'A':
        return 1
    elif x == 'C':
        return 2
    elif x == 'B':
        return 3
sale['store_id'] = sale.Branch.apply(store_id)
sale.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,customer_id,store_id
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08,Ewallet,522.83,4.761905,26.1415,9.1,1,1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29,Cash,76.40,4.761905,3.8200,9.6,3,2
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,13:23,Credit card,324.31,4.761905,16.2155,7.4,4,1
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,20:33,Ewallet,465.76,4.761905,23.2880,8.4,2,1
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,10:37,Ewallet,604.17,4.761905,30.2085,5.3,4,1


In [67]:
with open('inserting.sql', 'a') as f:
    m = range(data.shape[0])
    f.write("-- customer:\n\n")
    for t in data['Customer type'].unique():
        for g in data['Gender'].unique():
            f.write("INSERT INTO customer (customer_type, gender) VALUES('"+t+"', '"+g+"');\n")
    
    f.write("-- product:\n\n")
    for line in m:
        f.write("INSERT INTO product (product_line, unit_price) VALUES('"+data['Product line'][line]+"', "+str(data['Unit price'][line])+");\n")
    
    f.write("-- city:\n\n")
    for city in data['City'].unique():
        f.write("INSERT INTO city (city_name) VALUES('"+city+"');\n")
    
    f.write("-- branch:\n\n")
    for b in data['Branch'].unique():
        f.write("INSERT INTO branch (branch_letter) VALUES('"+b+"');\n")
         
    f.write("-- supermarket:\n\n")
    f.write("INSERT INTO supermarket (city_id, branch_id) VALUES(1, 1);\n")
    f.write("INSERT INTO supermarket (city_id, branch_id) VALUES(2, 2);\n")
    f.write("INSERT INTO supermarket (city_id, branch_id) VALUES(3, 3);\n")
    
    f.write("-- sale:\n\n")
    for line in m:
        c = str(sale['customer_id'][line])
        p = str(line+1)
        s = str(sale['store_id'][line])
        q = str(sale['Quantity'][line])
        ta = str(sale['Tax 5%'][line])
        t = str(sale['Total'][line])
        pa = sale['Payment'][line]
        co = str(sale['cogs'][line])
        gm = str(sale['gross margin percentage'][line])
        g = str(sale['gross income'][line])
        r = str(sale['Rating'][line])
        d = sale['Date'][line]
        ti = sale['Time'][line]
        f.write("INSERT INTO sale (customer_id,product_id,store_id,quantity,tax,total,payment_method,cogs,gross_margin_percentage,gross_income,rating,date,time) VALUES("+c+","+p+","+s+","+q+","+ta+","+t+",'"+pa+"',"+co+","+gm+","+g+","+r+",'"+d+"','"+ti+"');\n")
f.close()        
    